In [ ]:
# %%
#libs
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np


In [ ]:
# %%
# Exemplo: carregando o dataset
df = pd.read_csv("healthcare-dataset-stroke-data.csv")


In [ ]:
# %%
#dropar colunas desnecessarias
colunas_para_remover = ["id", "ever_married"]
df = df.drop(columns=colunas_para_remover)
print(df.head())

In [ ]:
#tratamento das colunas
#tratamento da coluna gender
df["gender"] = df["gender"].map({"Male": 0, "Female": 1})
#tratamento da coluna work_type colocando um valor pra cada
encoder = LabelEncoder()
df["work_type"] = encoder.fit_transform(df["work_type"])
#tratamento da coluna Residence_type
df["Residence_type"] = df["Residence_type"].map({"Urban": 0, "Rural": 1})
#remover todas os "N/A" da coluna bmi
df = df.dropna(subset=["bmi"])
#tratamento da coluna smoking_status
df["smoking_status"] = encoder.fit_transform(df["smoking_status"])

In [ ]:
# %%
na_count = df['bmi'].isna().sum()
print(f'Quantidade de N/A na coluna bmi: {na_count}')
#deletar linhas com valores nulos na coluna bmi
df.dropna(subset=['bmi'], inplace=True)


In [ ]:
# %%
# SEPARAR FEATURES E TARGET
X = df.drop('stroke', axis=1)  # Features (todas as colunas exceto 'stroke')
y = df['stroke']  # Target (coluna 'stroke')

In [ ]:
# %%
#serar a minha base de dados em 80% para treino e 20% para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#verificar a quantidade de dados em cada conjunto
print(f'Quantidade de dados no conjunto de treino: {X_train.shape[0]}')
print(f'Quantidade de dados no conjunto de teste: {X_test.shape[0]}')


In [ ]:
# %%
# CRIAR E TREINAR O MODELO
print("=== TREINANDO MODELO RANDOM FOREST ===")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# %%
# FAZER PREDIÇÕES E CALCULAR ACURÁCIA
y_pred = model.predict(X_test)

# Calcular e imprimir a acurácia
acuracia = accuracy_score(y_test, y_pred)
print(f'\n=== RESULTADOS DO MODELO ===')
print(f'Acurácia do modelo: {acuracia:.4f} ({acuracia*100:.2f}%)')

# Relatório de classificação mais detalhado
print('\nRelatório de Classificação:')
print(classification_report(y_test, y_pred))

In [ ]:
# %%
# FAZER PREDIÇÕES DE PROBABILIDADE
probabilidades = model.predict_proba(X_test)
# Probabilidades da classe 1 (ter AVC)
probabilidades_avc = probabilidades[:, 1]

print('\n=== PROBABILIDADES DE AVC (primeiras 10 amostras) ===')
for i in range(min(10, len(probabilidades_avc))):
    print(f'Amostra {i+1}: {probabilidades_avc[i]:.4f} ({probabilidades_avc[i]*100:.2f}%)')

In [ ]:
# FUNÇÃO PARA PREDIÇÃO INDIVIDUAL
def prever_risco_avc(modelo, dados_paciente):
    """
    Função para prever o risco de AVC de uma pessoa
    
    Parâmetros:
    - modelo: modelo treinado
    - dados_paciente: dicionário com os valores das features
    
    Retorna:
    - probabilidade de AVC em porcentagem
    """
    
    # Ordem das features que o modelo espera
    features_ordenadas = ['gender', 'age', 'hypertension', 'heart_disease', 
                         'work_type', 'Residence_type', 'avg_glucose_level', 
                         'bmi', 'smoking_status']
    
    # Criar array na ordem correta
    dados_array = [dados_paciente[feature] for feature in features_ordenadas]
    dados_array = np.array(dados_array).reshape(1, -1)
    
    # Fazer predição de probabilidade
    probabilidade = modelo.predict_proba(dados_array)[0, 1]
    
    return probabilidade * 100


In [ ]:
# EXEMPLO DE USO
print('\n=== EXEMPLO DE PREDIÇÃO INDIVIDUAL ===')

# Dados de exemplo de um paciente
paciente_exemplo = {
    'gender': 1,           # Female
    'age': 65,
    'hypertension': 1,     # Tem hipertensão
    'heart_disease': 0,    # Não tem doença cardíaca
    'work_type': 2,        # Private
    'Residence_type': 0,   # Urban
    'avg_glucose_level': 210.0,
    'bmi': 32.5,
    'smoking_status': 2    # Formerly smoked
}

risco_avc = prever_risco_avc(model, paciente_exemplo)
print(f'Probabilidade de AVC para o paciente: {risco_avc:.2f}%')

In [ ]:
# %%
# COMPARAR COM VALORES REAIS (para verificação)
print('\n=== COMPARAÇÃO COM VALORES REAIS (primeiras 5 amostras) ===')
print('Amostra | Probabilidade AVC | Predição | Real')
print('-' * 50)
for i in range(min(5, len(y_test))):
    prob = probabilidades_avc[i] * 100
    pred = y_pred[i]
    real = y_test.iloc[i]
    print(f'{i+1:6} | {prob:16.2f}% | {pred:8} | {real:4}')